In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install findspark
!pip install pyspark

--2021-11-02 12:00:06--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224445805 (214M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop2.7.tgz’

spark-3.1.2-bin-had 100%[===================>] 214.05M   168MB/s    in 1.3s    

2021-11-02 12:00:07 (168 MB/s) - ‘spark-3.1.2-bin-hadoop2.7.tgz’ saved [224445805/224445805]

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=1cc35739223a68a0696a52c3877cb4ebe67777afe9d9c7cf2bb52dd97047518a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
!pip install numpy


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext


sc = SparkContext("local[2]","page_block")

input = sc.textFile('./drive/MyDrive/pagerank_data.txt')

In [3]:
data = input.map(lambda x: [int(temp) for temp in x.split(' ')]).map(lambda x: (x[0],x[1:]))

In [12]:
# referenced_order =  sc.broadcast(input.map(lambda x: [(int(temp),1) for temp in x.split(' ')]).flatMap(lambda x: x).reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1],ascending=False).map(lambda x:x[0]).collect())
# data = input.map(lambda x: [int(temp) for temp in x.split(' ')]).map(lambda x: (x[0],sorted(x[1:],key=lambda y:referenced_order.value.index(y))))

In [6]:
def generate_block(N):
  temp = data
  for i in range(N):
    if(i==0):
      temp = data.map(lambda x: (x[0],x[1][:(len(x[1])//N)]))
    elif (i==N-1):
      temp += data.map(lambda x: (x[0],x[1][i*(len(x[1])//N):]))
    else:
      temp += data.map(lambda x: (x[0],x[1][i*(len(x[1])//N):(i+1)*(len(x[1])//N)]))
  return temp

In [9]:
def cul(N,maxIter):
  convergenceTol = 0.0001
  d= 0.85
  block_arr = generate_block(N)
  pr = sc.broadcast([1./float(30000) for _ in range(30000)])
  for _ in range(maxIter):
    iter = block_arr.flatMap(lambda t : [(target,(pr.value[t[0]]) / ((len(t[1])*N))) for target in t[1]])\
            .reduceByKey(lambda x,y : x+y).mapValues(lambda v : d*v).cache()

    temp = data.map(lambda x: (x[0],1/(30000)*(1-d)))
    result = temp.union(iter).reduceByKey(lambda x,y:x+y).sortByKey()
    err = result.map(lambda t: abs(pr.value[t[0]] - t[1])).sum()/N
    print(err)
    if err < convergenceTol:
      pr = result.map(lambda t:t[1]).collect()
      return pr
    pr = sc.broadcast(result.map(lambda t:t[1]).collect())
  return pr


In [ ]:
result = cul(1,1)

In [ ]:
result.value[:30]